In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


철강데이터가 위치한 폴더로 이동

In [ ]:
cd drive/MyDrive/SteelData/severstal-steel-defect-detection/

/content/drive/MyDrive/SteelData/severstal-steel-defect-detection


In [ ]:
ls

crop_defect2/  model_state_dict/      test_images/  train_image/
dataset/       new_train.csv          train/        train_log/
defect_class/  sample_submission.csv  train.csv


train폴더의 전체 이미지 파일 중 'train.csv' 파일의 ClassId가 i인 이미지만 추출한다.

단, 한 이미지에 여러 클래스가 있는 경우 학습이 제대로 되지 않기 때문에 gropby를 통해 같은 ImageId를 가진 파일 중 오로지 ClassId i만을 가진 이미지 파일만 추출하게 한다.

In [ ]:
import os
import pandas as pd
import shutil

# CSV 파일 경로
csv_file = 'train.csv'

# 이미지 파일이 있는 디렉토리 경로
image_dir = 'train/train_images/'

# 클래스 ID로 필터링
class_id_to_extract = 1

# 결과를 저장할 디렉토리 경로
output_dir = 'defect_class/defect1/'

# 결과 디렉토리 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# CSV 파일 읽기
data = pd.read_csv(csv_file)

# ImageId별로 그룹화하고 각 그룹에서 ClassId 값들의 집합 생성
grouped_data = data.groupby('ImageId')['ClassId'].apply(set)

for image_filename, class_ids in grouped_data.iteritems():
    # 해당 이미지가 오직 class_id_to_extract 결함만 가진 경우 복사
    if class_ids == {class_id_to_extract}:
        src_path = os.path.join(image_dir, image_filename)
        dst_path = os.path.join(output_dir, image_filename)

        # 이미지 파일 복사
        shutil.copy(src_path, dst_path)

print(f"{class_id_to_extract} 클래스의 이미지를 {output_dir} 디렉토리로 복사했습니다.")


<ipython-input-6-06d08e8c9475>:27: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for image_filename, class_ids in grouped_data.iteritems():


1 클래스의 이미지를 defect_class/defect1/ 디렉토리로 복사했습니다.


CSV파일에서 중복 클래스를 필터로 제거하면 아래 코드로 돌려도 된다.
아래 코드가 실행시간이 훨씬 짧다.

In [ ]:
# 전체 이미지에서 결함1번 클래스 이미지 추출
import os
import pandas as pd
import shutil

# CSV 파일 경로
csv_file = 'new_train.csv'

# 이미지 파일이 있는 디렉토리 경로
image_dir = 'train/train_images/'

# 클래스 ID로 필터링
class_id_to_extract = 3

# 결과를 저장할 디렉토리 경로
output_dir = 'defect_class/defect3/'

# 결과 디렉토리 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# CSV 파일 읽기
data = pd.read_csv(csv_file)

# classID가 2인 이미지 파일 복사
for index, row in data.iterrows():
    if row['ClassId'] == class_id_to_extract:
        image_filename = row['ImageId']  # 이미지 파일 이름
        src_path = os.path.join(image_dir, image_filename)
        dst_path = os.path.join(output_dir, image_filename)

        # 이미지 파일 복사
        shutil.copy(src_path, dst_path)

print(f"{class_id_to_extract} 클래스의 이미지를 {output_dir} 디렉토리로 복사했습니다.")


3 클래스의 이미지를 defect_class/defect3/ 디렉토리로 복사했습니다.


1600*256의 철강이미지 중 많은 부분이 검은색 배경을 가지고 있다. 학습을 위해 resize할때 유의미한 부분이 유실되지 않도록 6등분하고, 이 중 검은배경만 있는 이미지는 제외하고 저장한다.

In [ ]:
#6등분 후 검은배경만 있는 이미지 제외하고 저장
import os
from PIL import Image
import numpy as np

def divide_and_save_images(img_dir, save_dir):
    image_names = os.listdir(img_dir)
    threshold = 10  # 검정이미지로 판별할 임계값을 설정한다.

    for idx in range(len(image_names)):
        img_name = os.path.join(img_dir, image_names[idx])
        image = Image.open(img_name).convert('L') #grayscale로 이미지를 불러온다.

        # 이미지를 가로로 6등분하여 각각 (256, 256)로 리사이즈
        width, height = image.size
        part_width = width // 6

        for i in range(6):
            left = i * part_width
            right = left + part_width

            img_part= image.crop((left, 0,right ,height)).resize((256 ,256))

            # 임계값보다 높은 이미지만 저장한다. -> 임계값 보다 낮은 이미지는 검정 이미지로 판별해 저장하지 않는다.
            if np.array(img_part).mean() > threshold:
                img_part.save(os.path.join(save_dir, f"{image_names[idx].split('.')[0]}_{i}.jpg"))

divide_and_save_images('defect_class/defect3', 'defect_class/crop_defect3/defect3_cropped')


PGGAN 학습을 위한 데이터 전처리.

trian : 학습과 val : 검증데이터로 나눈다. 아래는 8:2로 기본 설정되어 있다.



In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 원본 이미지가 저장된 디렉토리
src_dir = "dataset/defect2"

# 훈련 세트와 검증 세트가 저장될 디렉토리
train_dir = "dataset/train/defect2"
val_dir = "dataset/val/defect2"

# 원본 이미지 파일 리스트 생성
file_list = os.listdir(src_dir)

# 파일 리스트를 훈련 리스트와 검증 리스트로 분할 (비율 8:2)
train_list, val_list = train_test_split(file_list, test_size=0.2)

# 각각의 리스트에 대한 파일을 train과 val디렉터리 내 defect폴더로 이동

for file_name in train_list:
    shutil.copy(os.path.join(src_dir, file_name), os.path.join(train_dir, file_name))

for file_name in val_list:
    shutil.copy(os.path.join(src_dir, file_name), os.path.join(val_dir, file_name))
